# Week 7: Agentic Frameworks - Multi-Agent Systems & Collaboration

## MBA 590 - Advanced AI Strategy: Prompting and Agentic Frameworks

---

## Overview

This week builds on our understanding of single agents by exploring Multi-Agent Systems (MAS) - where multiple autonomous agents work together. We'll examine communication protocols, coordination strategies, and how multiple agents can solve complex problems that would be difficult for a single agent.

### Key Topics
- Multi-Agent Systems (MAS) fundamentals
- Agent communication protocols and languages
- Coordination and collaboration strategies
- Cooperative vs. competitive agent interactions
- Business applications of multi-agent systems

## Learning Objectives

By the end of this week, you will be able to:

1. Understand the fundamentals of Multi-Agent Systems (MAS)
2. Design communication protocols for agent collaboration
3. Implement coordination strategies for multi-agent workflows
4. Identify scenarios where multi-agent approaches add value
5. Recognize coordination challenges and mitigation strategies
6. Apply multi-agent concepts to complex business processes

## Academic Readings

1. **Dorri, A., Kanhere, S. S., & Jurdak, R. (2018).** *Multi-Agent Systems: A Survey.* IEEE Access, 6, 28573-28593.

2. **Chen, L., Zaharia, M., & Zou, J. (2023).** *AutoGen: Enabling Next-Gen LLM Applications via Multi-Agent Conversation Framework.* arXiv preprint arXiv:2308.08155.

In [ ]:
# Setup: Import required libraries
import numpy as np
import pandas as pd
from typing import List, Dict, Tuple, Optional
import json
from dataclasses import dataclass, field
from enum import Enum
from datetime import datetime

print("Libraries imported successfully")

## 1. What are Multi-Agent Systems?

### Definition

A **Multi-Agent System (MAS)** consists of multiple autonomous agents that:
- Operate in a shared environment
- Interact with each other
- Pursue individual and/or collective goals
- Can coordinate, cooperate, or compete

### Why Use Multiple Agents?

1. **Specialization**: Different agents can have different expertise
2. **Scalability**: Distribute work across multiple agents
3. **Robustness**: System continues if one agent fails
4. **Modularity**: Easier to develop, test, and maintain
5. **Perspective**: Multiple viewpoints improve decision quality
6. **Natural Mapping**: Mirrors organizational structure

In [ ]:
# Comparison: Single vs. Multi-Agent

comparison = {
    'Aspect': [
        'Complexity',
        'Specialization',
        'Scalability',
        'Robustness',
        'Communication Overhead',
        'Coordination Effort',
        'Development Complexity',
        'Best For'
    ],
    'Single Agent': [
        'Lower',
        'Generalist approach',
        'Limited by single agent capacity',
        'Single point of failure',
        'None',
        'Not needed',
        'Simpler',
        'Focused, bounded tasks'
    ],
    'Multi-Agent': [
        'Higher',
        'Each agent can specialize',
        'Can distribute work',
        'Continues if one fails',
        'Significant',
        'Critical requirement',
        'More complex',
        'Complex, distributed tasks'
    ]
}

df_comparison = pd.DataFrame(comparison)
print("SINGLE AGENT vs. MULTI-AGENT SYSTEMS")
print("="*70)
print(df_comparison.to_string(index=False))

## 2. Agent Communication

### Communication Fundamentals

Agents need to exchange:
- **Information**: Data, facts, observations
- **Requests**: Ask other agents to perform actions
- **Responses**: Answer requests or queries
- **Coordination**: Synchronize activities
- **Negotiation**: Resolve conflicts or allocate resources

### Message Structure

Typical agent messages include:
- **Sender**: Who sent the message
- **Receiver**: Who should receive it
- **Performative**: Message type (inform, request, agree, refuse)
- **Content**: The actual message payload
- **Context**: Any relevant metadata

In [ ]:
# Agent message classes

class MessageType(Enum):
    """Types of messages agents can send."""
    INFORM = "inform"
    REQUEST = "request"
    QUERY = "query"
    AGREE = "agree"
    REFUSE = "refuse"
    PROPOSE = "propose"
    CONFIRM = "confirm"

@dataclass
class AgentMessage:
    """Structure for inter-agent communication."""
    sender: str
    receiver: str
    message_type: MessageType
    content: Dict
    timestamp: str = field(default_factory=lambda: datetime.now().isoformat())
    conversation_id: Optional[str] = None
    
    def to_dict(self) -> Dict:
        return {
            'sender': self.sender,
            'receiver': self.receiver,
            'type': self.message_type.value,
            'content': self.content,
            'timestamp': self.timestamp,
            'conversation_id': self.conversation_id
        }

# Example messages
msg1 = AgentMessage(
    sender="MarketingAgent",
    receiver="DataAgent",
    message_type=MessageType.REQUEST,
    content={"action": "get_customer_segments", "filters": {"region": "Northeast"}},
    conversation_id="conv_001"
)

msg2 = AgentMessage(
    sender="DataAgent",
    receiver="MarketingAgent",
    message_type=MessageType.INFORM,
    content={"segments": ["Enterprise", "SMB", "Startup"], "count": 3},
    conversation_id="conv_001"
)

print("AGENT COMMUNICATION EXAMPLES")
print("="*70)
print("\nMessage 1 (Request):")
print(json.dumps(msg1.to_dict(), indent=2))
print("\nMessage 2 (Response):")
print(json.dumps(msg2.to_dict(), indent=2))

## 3. Agent Coordination Strategies

### A. Centralized Coordination

A coordinator agent manages all other agents.

**Pros:**
- Simple to implement
- Clear authority
- Easier to maintain consistency

**Cons:**
- Single point of failure
- Bottleneck for communication
- Less scalable

In [ ]:
# Centralized coordination example

class CoordinatorAgent:
    """Central coordinator managing multiple worker agents."""
    
    def __init__(self, name: str):
        self.name = name
        self.workers = {}
        self.tasks = []
        self.message_log = []
    
    def register_worker(self, agent_name: str, capabilities: List[str]):
        """Register a worker agent and its capabilities."""
        self.workers[agent_name] = {
            'capabilities': capabilities,
            'status': 'idle',
            'current_task': None
        }
        print(f"Registered {agent_name} with capabilities: {capabilities}")
    
    def assign_task(self, task: Dict) -> Optional[str]:
        """Assign task to appropriate worker."""
        required_capability = task['requires']
        
        # Find capable and idle worker
        for agent_name, info in self.workers.items():
            if (required_capability in info['capabilities'] and 
                info['status'] == 'idle'):
                
                # Assign task
                info['status'] = 'busy'
                info['current_task'] = task
                
                # Send message
                msg = AgentMessage(
                    sender=self.name,
                    receiver=agent_name,
                    message_type=MessageType.REQUEST,
                    content=task
                )
                self.message_log.append(msg)
                
                print(f"Assigned task '{task['description']}' to {agent_name}")
                return agent_name
        
        print(f"No available agent for task '{task['description']}'")
        return None
    
    def complete_task(self, agent_name: str, result: Dict):
        """Mark task as complete and free the worker."""
        if agent_name in self.workers:
            self.workers[agent_name]['status'] = 'idle'
            self.workers[agent_name]['current_task'] = None
            print(f"{agent_name} completed task")

# Demo centralized coordination
coordinator = CoordinatorAgent("MainCoordinator")

print("\nCENTRALIZED COORDINATION DEMO")
print("="*70)

# Register workers
coordinator.register_worker("DataAnalyst", ['data_analysis', 'statistics'])
coordinator.register_worker("ResearchAgent", ['research', 'synthesis'])
coordinator.register_worker("WriterAgent", ['writing', 'editing'])

print("\nAssigning tasks...")
# Assign tasks
coordinator.assign_task({
    'description': 'Analyze Q4 sales data',
    'requires': 'data_analysis'
})

coordinator.assign_task({
    'description': 'Research market trends',
    'requires': 'research'
})

coordinator.assign_task({
    'description': 'Write executive summary',
    'requires': 'writing'
})

### B. Decentralized Coordination

Agents coordinate directly with each other without a central authority.

**Pros:**
- More scalable
- No single point of failure
- Flexible adaptation

**Cons:**
- More complex communication
- Harder to maintain consistency
- Potential for conflicts

In [ ]:
# Decentralized coordination example

class DecentralizedAgent:
    """Agent that coordinates directly with peers."""
    
    def __init__(self, name: str, capabilities: List[str]):
        self.name = name
        self.capabilities = capabilities
        self.peers = {}
        self.inbox = []
        self.status = 'idle'
    
    def discover_peer(self, peer_name: str, capabilities: List[str]):
        """Learn about another agent."""
        self.peers[peer_name] = capabilities
    
    def request_help(self, capability_needed: str, task: str) -> Optional[str]:
        """Find a peer who can help with a task."""
        for peer_name, capabilities in self.peers.items():
            if capability_needed in capabilities:
                msg = AgentMessage(
                    sender=self.name,
                    receiver=peer_name,
                    message_type=MessageType.REQUEST,
                    content={'task': task, 'capability': capability_needed}
                )
                print(f"{self.name} → {peer_name}: REQUEST '{task}'")
                return peer_name
        return None
    
    def respond_to_request(self, sender: str, can_help: bool):
        """Respond to a peer's request."""
        msg_type = MessageType.AGREE if can_help else MessageType.REFUSE
        print(f"{self.name} → {sender}: {msg_type.value.upper()}")
        return msg_type

# Demo decentralized coordination
print("\nDECENTRALIZED COORDINATION DEMO")
print("="*70)

# Create agents
agent_a = DecentralizedAgent("AnalysisAgent", ['data_analysis'])
agent_b = DecentralizedAgent("ResearchAgent", ['research'])
agent_c = DecentralizedAgent("WritingAgent", ['writing'])

# Agents discover each other
print("\nPeer Discovery:")
agent_a.discover_peer("ResearchAgent", ['research'])
agent_a.discover_peer("WritingAgent", ['writing'])
agent_b.discover_peer("AnalysisAgent", ['data_analysis'])
agent_b.discover_peer("WritingAgent", ['writing'])
print("Agents discovered their peers")

# Agents request help from each other
print("\nDirect Coordination:")
agent_a.request_help('research', 'Find industry benchmarks')
agent_b.respond_to_request('AnalysisAgent', can_help=True)

agent_b.request_help('data_analysis', 'Calculate market share')
agent_a.respond_to_request('ResearchAgent', can_help=True)

## 4. Multi-Agent Workflow Patterns

Common patterns for agent collaboration.

In [ ]:
# Multi-agent workflow patterns

workflow_patterns = {
    'Pattern': [
        'Sequential',
        'Parallel',
        'Hierarchical',
        'Marketplace',
        'Debate/Consensus',
        'Swarm'
    ],
    'Description': [
        'Agents work in sequence, passing results',
        'Agents work simultaneously on different parts',
        'Supervisor delegates to specialized workers',
        'Agents bid for tasks based on capability',
        'Agents discuss and reach consensus',
        'Many simple agents collaborate emergently'
    ],
    'Best For': [
        'Pipeline processing, document workflows',
        'Independent subtasks, scalability',
        'Complex projects with clear structure',
        'Resource optimization, load balancing',
        'Decision-making, quality improvement',
        'Exploration, optimization problems'
    ],
    'Example': [
        'Research → Analyze → Write → Review',
        'Multiple analysts each handle one region',
        'PM coordinates designers, developers, testers',
        'Tasks assigned to lowest-cost capable agent',
        'Multiple experts debate best strategy',
        'Many agents explore solution space'
    ]
}

df_patterns = pd.DataFrame(workflow_patterns)
print("\nMULTI-AGENT WORKFLOW PATTERNS")
print("="*70)
for idx, row in df_patterns.iterrows():
    print(f"\n{row['Pattern']}:")
    print(f"  Description: {row['Description']}")
    print(f"  Best For: {row['Best For']}")
    print(f"  Example: {row['Example']}")

## 5. Business Multi-Agent Application: Product Launch

Let's design a multi-agent system for coordinating a product launch.

In [ ]:
# Product launch multi-agent system

class ProductLaunchAgent:
    """Base class for product launch agents."""
    
    def __init__(self, name: str, role: str):
        self.name = name
        self.role = role
        self.tasks = []
        self.status = "ready"
    
    def receive_task(self, task: Dict):
        """Receive a task from another agent."""
        self.tasks.append(task)
        print(f"{self.name} ({self.role}) received: {task['description']}")
    
    def complete_task(self, task_id: int) -> Dict:
        """Complete a task and return results."""
        if task_id < len(self.tasks):
            task = self.tasks[task_id]
            result = {
                'task': task['description'],
                'completed_by': self.name,
                'status': 'completed'
            }
            print(f"{self.name} completed: {task['description']}")
            return result
        return {}

class ProductLaunchCoordinator:
    """Coordinates the product launch across multiple teams."""
    
    def __init__(self):
        self.agents = {}
        self.timeline = []
    
    def add_agent(self, agent: ProductLaunchAgent):
        """Add an agent to the launch team."""
        self.agents[agent.role] = agent
    
    def coordinate_launch(self, launch_date: str):
        """Coordinate tasks across all agents."""
        print(f"\nCoordinating product launch for {launch_date}")
        print("="*70)
        
        # Define launch tasks and assign to appropriate agents
        tasks = [
            {'agent': 'marketing', 'description': 'Create launch campaign', 'week': -8},
            {'agent': 'product', 'description': 'Finalize product features', 'week': -12},
            {'agent': 'sales', 'description': 'Train sales team', 'week': -4},
            {'agent': 'marketing', 'description': 'Prepare press release', 'week': -2},
            {'agent': 'product', 'description': 'Complete quality testing', 'week': -6},
            {'agent': 'sales', 'description': 'Set up CRM campaign', 'week': -3}
        ]
        
        # Sort by week
        tasks.sort(key=lambda x: x['week'])
        
        # Assign tasks
        print("\nTask Assignment (weeks before launch):")
        for task in tasks:
            agent_role = task['agent']
            if agent_role in self.agents:
                self.agents[agent_role].receive_task(task)
        
        return len(tasks)

# Demo product launch coordination
print("\nPRODUCT LAUNCH MULTI-AGENT SYSTEM")
print("="*70)

# Create specialized agents
marketing_agent = ProductLaunchAgent("MarketingLead", "marketing")
product_agent = ProductLaunchAgent("ProductManager", "product")
sales_agent = ProductLaunchAgent("SalesDirector", "sales")

# Create coordinator
coordinator = ProductLaunchCoordinator()
coordinator.add_agent(marketing_agent)
coordinator.add_agent(product_agent)
coordinator.add_agent(sales_agent)

# Coordinate launch
coordinator.coordinate_launch("2025-03-01")

print("\nSimulating task completion:")
product_agent.complete_task(0)
marketing_agent.complete_task(0)
sales_agent.complete_task(0)

## 6. Coordination Challenges

Common challenges in multi-agent systems and solutions.

In [ ]:
# Coordination challenges and solutions

challenges = {
    'Challenge': [
        'Message Overload',
        'Deadlock',
        'Conflicting Goals',
        'Information Consistency',
        'Task Allocation',
        'Trust and Security'
    ],
    'Description': [
        'Too many messages, communication bottleneck',
        'Agents waiting on each other indefinitely',
        'Agents optimizing for different objectives',
        'Different agents have inconsistent information',
        'Inefficient or unfair task distribution',
        'Malicious or malfunctioning agents'
    ],
    'Mitigation': [
        'Message filtering, priority queues, async communication',
        'Timeouts, deadlock detection, hierarchical coordination',
        'Align incentives, shared objectives, negotiation protocols',
        'Shared knowledge base, synchronization points, versioning',
        'Capability matching, auction mechanisms, load balancing',
        'Authentication, authorization, reputation systems, monitoring'
    ],
    'Business Impact': [
        'Slow response time, high costs',
        'System hangs, missed deadlines',
        'Suboptimal outcomes, wasted resources',
        'Incorrect decisions, errors',
        'Inefficiency, agent idle time',
        'Security breaches, system compromise'
    ]
}

df_challenges = pd.DataFrame(challenges)
print("\nMULTI-AGENT COORDINATION CHALLENGES")
print("="*70)
for idx, row in df_challenges.iterrows():
    print(f"\n{idx + 1}. {row['Challenge']}")
    print(f"   Problem: {row['Description']}")
    print(f"   Solution: {row['Mitigation']}")
    print(f"   Impact: {row['Business Impact']}")

## 7. Business Applications of Multi-Agent Systems

In [ ]:
# Business applications

applications = {
    'Domain': [
        'Supply Chain',
        'Customer Service',
        'Financial Trading',
        'Project Management',
        'Business Intelligence'
    ],
    'Agent Types': [
        'Supplier agents, Inventory agents, Logistics agents',
        'Routing agent, FAQ agent, Escalation agent, Follow-up agent',
        'Market analysis agent, Risk agent, Execution agent',
        'Planning agent, Resource agent, Monitoring agent, Reporting agent',
        'Data collection agents, Analysis agents, Visualization agent'
    ],
    'Coordination Need': [
        'Optimize inventory levels, minimize delays, balance costs',
        'Route to right agent, maintain context, coordinate responses',
        'Share market data, coordinate strategies, manage risk',
        'Allocate resources, track progress, identify bottlenecks',
        'Gather from multiple sources, integrate analyses, create reports'
    ],
    'Business Value': [
        'Reduced costs, faster delivery, better reliability',
        'Faster resolution, higher satisfaction, 24/7 coverage',
        'Better returns, risk management, faster execution',
        'On-time delivery, resource efficiency, early warning',
        'Comprehensive insights, faster decisions, better accuracy'
    ]
}

print("\nBUSINESS APPLICATIONS OF MULTI-AGENT SYSTEMS")
print("="*70)
for i, domain in enumerate(applications['Domain']):
    print(f"\n{domain}:")
    print(f"  Agents: {applications['Agent Types'][i]}")
    print(f"  Coordination: {applications['Coordination Need'][i]}")
    print(f"  Value: {applications['Business Value'][i]}")

## 8. Hands-On Practice Activity

### Design a Multi-Agent System for Your Business

In [ ]:
# YOUR TURN: Design your multi-agent system

my_multi_agent_design = """
BUSINESS PROCESS:
[Describe a complex cross-functional process]

Example: Coordinating a product launch between marketing, sales, and development

AGENT ROSTER:
1. Agent Name: [Name]
   Role: [Responsibility]
   Capabilities: [What it can do]
   
2. Agent Name:
   Role:
   Capabilities:
   
[Add more agents...]

COORDINATION STRATEGY:
[Centralized, Decentralized, or Hybrid? Why?]

WORKFLOW PATTERN:
[Sequential, Parallel, Hierarchical, etc.? Why?]

COMMUNICATION PROTOCOL:
[How will agents share information?]

COORDINATION CHALLENGES:
[What challenges do you anticipate?]

MITIGATION STRATEGIES:
[How will you address those challenges?]
"""

print(my_multi_agent_design)

In [ ]:
# YOUR TURN: Map out interactions

my_agent_interactions = [
    {
        'from': 'Agent A',
        'to': 'Agent B',
        'message_type': 'request/inform/query',
        'content': 'What information is shared',
        'purpose': 'Why this communication is needed'
    },
    # Add more interactions...
]

print("MY AGENT INTERACTIONS:")
for i, interaction in enumerate(my_agent_interactions, 1):
    print(f"\nInteraction {i}:")
    print(f"  {interaction['from']} → {interaction['to']}")
    print(f"  Type: {interaction['message_type']}")
    print(f"  Content: {interaction['content']}")
    print(f"  Purpose: {interaction['purpose']}")

## 9. Discussion Questions

Reflect on the following:

1. **Multi-Agent Design**: Design a simple multi-agent system for a business task (e.g., coordinating a product launch between marketing, sales, and development agents). What information would need to be shared between agents? What coordination challenges might arise?

2. **Coordination vs. Autonomy**: How do you balance agent autonomy with the need for coordination? Where should you allow independent decision-making vs. requiring consensus?

3. **Communication Overhead**: As you add more agents, communication overhead grows. How do you decide the optimal number of agents for a task?

4. **Conflict Resolution**: When agents have conflicting goals or recommendations, how should conflicts be resolved? Who or what should have final authority?

5. **Failure Handling**: In a multi-agent system, what happens if one agent fails or provides incorrect information? How does this propagate?

6. **Human Oversight**: Where in a multi-agent workflow should humans be involved? Which decisions should remain human-only?

7. **ROI Considerations**: Multi-agent systems are more complex than single agents. When is this complexity justified by the business value?

### Your Reflections:

**Question 1 - Multi-Agent Design:**

[Your response]

**Question 2 - Coordination vs. Autonomy:**

[Your response]

**Question 3 - Communication Overhead:**

[Your response]

**Question 4 - Conflict Resolution:**

[Your response]

**Question 5 - Failure Handling:**

[Your response]

**Question 6 - Human Oversight:**

[Your response]

**Question 7 - ROI Considerations:**

[Your response]

## 10. Key Takeaways

1. **Multi-agent systems enable specialization and scalability** through division of labor and parallel processing

2. **Effective communication protocols are essential** for agent coordination and collaboration

3. **Coordination strategies vary** from centralized (simple, less scalable) to decentralized (complex, more robust)

4. **Different workflow patterns serve different needs** - sequential, parallel, hierarchical, marketplace, debate, or swarm

5. **Coordination challenges are significant** - message overload, deadlock, conflicts, consistency, allocation, and security

6. **Business value comes from handling complex cross-functional processes** that mirror organizational structure

7. **Complexity must be justified** - only use multi-agent systems when the benefits outweigh the coordination overhead

## 11. Looking Ahead to Week 8

Next week, we'll explore **Agentic Frameworks: Business Applications & Case Studies**.

We'll cover:
- Real-world applications in customer service, data analysis, supply chain
- Case studies of successful agentic system deployments
- Implementation challenges and lessons learned
- Evaluating when to deploy agentic solutions

**Assignment 1 Due**: Advanced Prompt Engineering Analysis

**Preparation:** Reflect on your organization's most complex processes. Which might benefit from agentic systems?

## Additional Resources

### Frameworks:
- [AutoGen (Microsoft)](https://github.com/microsoft/autogen) - Multi-agent conversation framework
- [CrewAI](https://github.com/joaomdmoura/crewAI) - Framework for orchestrating role-playing agents
- [LangGraph](https://github.com/langchain-ai/langgraph) - Build multi-agent workflows as graphs

### Academic Resources:
- [Multi-Agent Systems Survey (Dorri et al., 2018)](https://ieeexplore.ieee.org/document/8387658)
- [AutoGen Paper (Chen et al., 2023)](https://arxiv.org/abs/2308.08155)

### Practical Guides:
- [Building Multi-Agent Systems with LangChain](https://python.langchain.com/docs/use_cases/multi_agent/)
- [Anthropic Multi-Agent Patterns](https://docs.anthropic.com/claude/docs/)

---

*End of Week 7 Notebook*